<a href="https://colab.research.google.com/github/aatika-hakim/Langgraph/blob/main/Music_Compositor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain langchain_groq langchain_openai streamlit langgraph music21 python-dotenv PyPDF2 pygame

  Using cached httpx-0.28.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.28.0-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 3.0.0 requires httpx=

In [4]:
!pip install googletrans

  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groq 0.13.0 requires htt

In [5]:
!pip install --upgrade httpx groq

  Using cached httpx-0.28.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.28.0-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
ERROR: Operation cancelled by user


In [ ]:
import streamlit as st
import requests
from langchain_groq import ChatGroq
from googletrans import Translator
import plotly.express as px
from google.colab import userdata

api_key: str = userdata.get('GROQ_API_KEY')
NEWS_API_KEY:str = userdata.get('NEWS_API_KEY')

# Initialize services


# Initialize the Language Model (LLM) for generating musical components
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0, api_key=api_key)
translator = Translator()

# Reliability grading function
def grade_reliability(source):
    high_reliability = ["bbc", "reuters", "apnews"]
    medium_reliability = ["cnn", "nytimes", "theguardian"]
    low_reliability = ["buzzfeed", "unknown"]

    for keyword in high_reliability:
        if keyword in source.lower():
            return 5
    for keyword in medium_reliability:
        if keyword in source.lower():
            return 3
    return 1

# Fetch news articles
def fetch_news(topic):
    url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        st.error("Error fetching news!")
        return []

# Summarize news
def summarize_news(articles, length="short", tone="formal"):
    content = [article.get("content", "") for article in articles[:5]]
    prompt = f"Summarize the following news articles in a {length} {tone} tone:\n\n{''.join(content)}"
    return llm.predict(prompt)

# Translate text
def translate_summary(summary, target_language):
    return translator.translate(summary, dest=target_language).text

# Initialize session state
if "query_count" not in st.session_state:
    st.session_state.query_count = 0
if "max_queries" not in st.session_state:
    st.session_state.max_queries = 3  # Free-tier limit

# Streamlit App
st.title("Advanced News Chatbot")
st.sidebar.header("Settings")
st.sidebar.info(f"Queries used: {st.session_state.query_count}/{st.session_state.max_queries}")

# News Topic Input
topic = st.text_input("Enter a topic to fetch news:", "AI")
length = st.selectbox("Select summary length:", ["short", "medium", "long"])
tone = st.selectbox("Select summary tone:", ["formal", "casual", "bullet points"])
language = st.selectbox("Translate summary to:", ["English", "Spanish", "French", "German", "None"])

# Fetch News Button
if st.button("Fetch News"):
    if st.session_state.query_count >= st.session_state.max_queries:
        st.warning("You've reached your daily limit! Upgrade for unlimited access.")
    else:
        with st.spinner("Fetching news..."):
            articles = fetch_news(topic)
        if articles:
            st.success(f"Fetched {len(articles)} articles on '{topic}'.")
            st.session_state.query_count += 1

            # Display articles with reliability scores
            for i, article in enumerate(articles[:5]):
                st.write(f"### {i + 1}. {article['title']} (Reliability: {grade_reliability(article.get('source', {}).get('name', 'unknown'))}/5)")
                st.write(article.get("description", "No description available."))
                st.write(f"[Read more]({article['url']})")

            # Summarize News Button
            if st.button("Summarize News"):
                with st.spinner("Summarizing news..."):
                    summary = summarize_news(articles, length, tone)
                    if language != "None":
                        summary = translate_summary(summary, language.lower())
                st.write("### Summary")
                st.write(summary)

# Premium upgrade message
st.sidebar.markdown("**Want unlimited queries and advanced features?**")

# Interactive Analytics (for Premium Users)
if st.sidebar.checkbox("Show Analytics (Premium)"):
    st.subheader("Trending Topics")
    topics = [article["title"] for article in articles[:10]]
    fig = px.bar(x=topics, y=[len(topic) for topic in topics], labels={"x": "Topic", "y": "Length of Title"})
    st.plotly_chart(fig)

# Toggle between Dark Mode and Light Mode
dark_mode = st.sidebar.checkbox("Enable Dark Mode")
if dark_mode:
    st.markdown(
        """
        <style>
        body {
            background-color: #1a1a1a;
            color: #ffffff;
        }
        </style>
        """,
        unsafe_allow_html=True,
    )


In [ ]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata
import music21
import tempfile
import os
import random
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key: str = userdata.get('GROQ_API_KEY')

# Initialize the Language Model (LLM) for generating musical components
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0, api_key=api_key)

# State Definition
class MusicState(TypedDict):
    """Define the structure of the state for the music generation workflow."""
    musician_input: str  # User's input describing the desired music
    melody: str          # Generated melody
    harmony: str         # Generated harmony
    rhythm: str          # Generated rhythm
    style: str           # Desired musical style
    composition: str     # Complete musical composition
    midi_file: str       # Path to the generated MIDI file

# Component Functions
def melody_generator(state: MusicState) -> Dict:
    """Generate a melody based on the user's input."""
    prompt = ChatPromptTemplate.from_template(
        "Generate a melody based on this input: {input}. Represent it as a string of notes in music21 format."
    )
    chain = prompt | llm
    melody = chain.invoke({"input": state["musician_input"]})
    return {"melody": melody.content}

def harmony_creator(state: MusicState) -> Dict:
    """Create harmony for the generated melody."""
    prompt = ChatPromptTemplate.from_template(
        "Create harmony for this melody: {melody}. Represent it as a string of chords in music21 format."
    )
    chain = prompt | llm
    harmony = chain.invoke({"melody": state["melody"]})
    return {"harmony": harmony.content}

def rhythm_analyzer(state: MusicState) -> Dict:
    """Analyze and suggest a rhythm for the melody and harmony."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze and suggest a rhythm for this melody and harmony: {melody}, {harmony}. Represent it as a string of durations in music21 format."
    )
    chain = prompt | llm
    rhythm = chain.invoke({"melody": state["melody"], "harmony": state["harmony"]})
    return {"rhythm": rhythm.content}

def style_adapter(state: MusicState) -> Dict:
    """Adapt the composition to the specified musical style."""
    prompt = ChatPromptTemplate.from_template(
        "Adapt this composition to the {style} style: Melody: {melody}, Harmony: {harmony}, Rhythm: {rhythm}. Provide the result in music21 format."
    )
    chain = prompt | llm
    adapted = chain.invoke({
        "style": state["style"],
        "melody": state["melody"],
        "harmony": state["harmony"],
        "rhythm": state["rhythm"]
    })
    return {"composition": adapted.content}

def midi_converter(state: MusicState) -> Dict:
    """Convert the composition to MIDI format and save it as a file."""
    piece = music21.stream.Score()

    # Create the melody and harmony from the state
    user_input = state['musician_input'].lower()
    scale_name = 'C major' if 'major' in user_input else 'C minor'

    melody = create_melody(scale_name, 7)
    harmony = create_chord_progression(7)

    piece.append(melody)
    piece.append(harmony)
    piece.insert(0, music21.tempo.MetronomeMark(number=60))

    # Use NamedTemporaryFile to create a temp MIDI file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        piece.write('midi', temp_midi.name)
        midi_file_path = temp_midi.name  # Get the path of the temp file

    return {"midi_file": midi_file_path}


# Graph Construction
workflow = StateGraph(MusicState)

# Add nodes to the graph
workflow.add_node("melody_generator", melody_generator)
workflow.add_node("harmony_creator", harmony_creator)
workflow.add_node("rhythm_analyzer", rhythm_analyzer)
workflow.add_node("style_adapter", style_adapter)
workflow.add_node("midi_converter", midi_converter)

# Set the entry point of the graph
workflow.set_entry_point("melody_generator")

# Add edges to connect the nodes
workflow.add_edge("melody_generator", "harmony_creator")
workflow.add_edge("harmony_creator", "rhythm_analyzer")
workflow.add_edge("rhythm_analyzer", "style_adapter")
workflow.add_edge("style_adapter", "midi_converter")
workflow.add_edge("midi_converter", END)

# Compile the graph
app = workflow.compile()

# Invocation Example
inputs = {
    "musician_input": "Create a relaxing melody with a classical style",
    "style": "Classical"
}

result = app.invoke(inputs)
print("MIDI file created at:", result["midi_file"])




In [ ]:

import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmpi1l5f6dw.mid', './music.mid')

In [ ]:
import os
from music21 import note, stream, tempo, chord
import tempfile
import random

scales = {
    'C major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'C minor': ['C', 'D', 'Eb', 'F', 'G', 'Ab', 'Bb'],
}

chords = {
    'C major': ['C4', 'E4', 'G4'],
    'C minor': ['C4', 'Eb4', 'G4'],
}

def create_melody(scale_name, duration):
    melody = stream.Part()
    scale = scales[scale_name]
    for _ in range(duration):
        note_obj = note.Note(random.choice(scale) + '4')
        note_obj.quarterLength = 1
        melody.append(note_obj)
    return melody

def create_chord_progression(duration):
    """Create a simple chord progression."""
    harmony = stream.Part()
    for _ in range(duration):
        chord_name = random.choice(list(chords.keys()))
        chord_obj = chord.Chord(chords[chord_name])  # Use chord.Chord instead of stream.Chord
        chord_obj.quarterLength = 1
        harmony.append(chord_obj)
    return harmony

def generate_midi_file(scale_name="C major", duration=7):
    piece = stream.Score()
    melody = create_melody(scale_name, duration)
    harmony = create_chord_progression(duration)
    piece.append(melody)
    piece.append(harmony)
    piece.insert(0, tempo.MetronomeMark(number=60))

    # Use a temporary file path
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        file_path = temp_midi.name

    try:
        piece.write('midi', file_path)
        print("MIDI file created at:", file_path)
    except Exception as e:
        print("Error while creating MIDI file:", e)

# Run the function
generate_midi_file(scale_name="C major", duration=7)


In [ ]:
import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmpqosx_5o_.mid', './your_music.mid')


In [ ]:
# Additional Imports
from typing import List

# Update MusicState to include lyrics and sections
class MusicState(TypedDict):
    musician_input: str
    style: str
    melody: str
    harmony: str
    rhythm: str
    lyrics: str
    composition: str
    midi_file: str

def lyrics_generator(state: MusicState) -> Dict:
    print("Generating lyrics...")
    prompt = ChatPromptTemplate.from_template(
        "Generate song lyrics based on this theme: {input}. Include verses and a chorus."
    )
    chain = prompt | llm
    lyrics = chain.invoke({"input": state["musician_input"]})
    print("Lyrics generated:", lyrics.content)
    return {"lyrics": lyrics.content}  # Updates 'lyrics'


def verse_creator(state: MusicState) -> Dict:
    print("Creating verse melody and harmony...")
    prompt = ChatPromptTemplate.from_template(
        "Generate a verse melody and harmony in {style} style for lyrics: {lyrics}. Use music21 format."
    )
    chain = prompt | llm
    verse = chain.invoke({"style": state["style"], "lyrics": state["lyrics"]})
    print("Verse created:", verse.content)
    # Ensure that 'melody' key is updated in MusicState
    return {"melody": verse.content}  # Updates 'melody'


def chorus_creator(state: MusicState) -> Dict:
    print("Creating chorus melody and harmony...")
    prompt = ChatPromptTemplate.from_template(
        "Generate a chorus melody and harmony in {style} style for lyrics: {lyrics}. Use music21 format."
    )
    chain = prompt | llm
    chorus = chain.invoke({"style": state["style"], "lyrics": state["lyrics"]})
    print("Chorus created:", chorus.content)
    # Ensure that 'harmony' key is updated in MusicState
    return {"harmony": chorus.content}  # Updates 'harmony'


def arrange_song(state: MusicState) -> Dict:
    print("Arranging song...")
    composition = f"Verse:\n{state['melody']}\nChorus:\n{state['harmony']}\n"
    print("Composition arranged:", composition)
    # Ensure that 'composition' key is updated in MusicState
    return {"composition": composition}  # Updates 'composition'


def midi_converter(state: MusicState) -> Dict:
    print("Converting to MIDI...")
    piece = music21.stream.Score()
    # Add sections
    verse_melody = create_melody('C major', 8)
    chorus_melody = create_melody('C major', 8)
    piece.append(verse_melody)
    piece.append(chorus_melody)
    piece.insert(0, music21.tempo.MetronomeMark(number=90))

    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        piece.write('midi', temp_midi.name)
        midi_file_path = temp_midi.name
    print("MIDI file created:", midi_file_path)

    # Ensure that 'midi_file' key is updated in MusicState
    return {"midi_file": midi_file_path}  # Updates 'midi_file'



# Revised Graph Construction
# Reinitialize the StateGraph and Compile it after the function changes
workflow = StateGraph(MusicState)

# Add nodes for the full song generation
workflow.add_node("lyrics_generator", lyrics_generator)
workflow.add_node("verse_creator", verse_creator)
workflow.add_node("chorus_creator", chorus_creator)
workflow.add_node("arrange_song", arrange_song)
workflow.add_node("midi_converter", midi_converter)

# Set the entry point and define edges
workflow.set_entry_point("lyrics_generator")
workflow.add_edge("lyrics_generator", "verse_creator")
workflow.add_edge("verse_creator", "chorus_creator")
workflow.add_edge("chorus_creator", "arrange_song")
workflow.add_edge("arrange_song", "midi_converter")
workflow.add_edge("midi_converter", END) # Connect the last node to the END

# Compile the graph
app = workflow.compile()  # This was missing in the previous response

inputs = {
    "musician_input": "A peaceful, soulful love song in a classical style",
    "style": "Classical"
}

result = app.invoke(inputs)
print("Song MIDI file created at:", result["midi_file"])


In [ ]:
import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmparrovzjy.mid', './song.mid')
